In [ ]:
!pip install --quiet robustness

In [2]:
import numpy as np
import torch
import torch.nn as nn
import os
import sys
import json
import pandas as pd

from robustness.tools.imagenet_helpers import ImageNetHierarchy, common_superclass_wnid

%reload_ext autoreload
%autoreload 2

In [3]:
class CustomImageNetHierarchy(ImageNetHierarchy):
    """
    Download metadata:
    $ curl -o imagenet_class_index.json https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json
    $ curl -o words.txt https://image-net.org/data/words.txt
    $ curl -o wordnet.is_a.txt https://image-net.org/data/wordnet.is_a.txt

    References:
        https://github.com/google-research/meta-dataset/issues/65
        https://github.com/MadryLab/robustness/blob/master/robustness/tools/imagenet_helpers.py
    """

    @staticmethod
    def load_imagenet_info(ds_path, ds_info_path):
        """
        Get information about mapping between ImageNet wnids/class numbers/class names.

        Args:
            ds_path (str) : Path to ImageNet dataset
            ds_info_path (str) : Path to supplementary files for the ImageNet dataset
                                 ('wordnet.is_a.txt', 'words.txt', 'imagenet_class_index.json')
                                 which can be obtained from http://image-net.org/download-API.

        """
        #files = os.listdir(os.path.join(ds_path, 'train'))
        #in_wnids = [f for f in files if f[0]=='n']

        f = open(os.path.join(ds_info_path, "words.txt"))
        wnid_to_name = [l.strip() for l in f.readlines()]
        wnid_to_name = {l.split("\t")[0]: l.split("\t")[1] for l in wnid_to_name}

        with open(os.path.join(ds_info_path, "imagenet_class_index.json"), "r") as f:
            base_map = json.load(f)
            wnid_to_num = {v[0]: int(k) for k, v in base_map.items()}
            num_to_name = {int(k): v[1] for k, v in base_map.items()}

        in_wnids = list(wnid_to_num.keys())

        return in_wnids, wnid_to_name, wnid_to_num, num_to_name

In [4]:
superclass_wnids = common_superclass_wnid("mixed_10")
hierarchy = CustomImageNetHierarchy(None, "data/imagenet-embeddings")
superclass_classes, superclass_names = hierarchy.get_subclasses(superclass_wnids, balanced=False)

superclass_names

{0: 'dog, domestic dog, Canis familiaris',
 1: 'bird',
 2: 'insect',
 3: 'monkey',
 4: 'car, auto, automobile, machine, motorcar',
 5: 'feline, felid',
 6: 'truck, motortruck',
 7: 'fruit',
 8: 'fungus',
 9: 'boat'}

In [16]:
other_classes = set(range(1_000)) - set.union(*superclass_classes)
superclass_classes += [other_classes]

class_map = {}

for i, superclass_classes_i in enumerate(superclass_classes):
    for _class in superclass_classes_i:
        class_map[_class] = i

In [18]:
n_classes = [len(classes) for classes in superclass_classes]
n_classes_frac = np.array(n_classes) / np.sum(n_classes)

table = {
    "superclass": [superclass_names[i] for i in sorted(superclass_names.keys())],
    "n_classes": n_classes,
    "n_classes_frac": n_classes_frac,
    "classes": [sorted(superclasses_i) for superclasses_i in superclass_classes],
}

table

{'superclass': ['dog, domestic dog, Canis familiaris',
  'bird',
  'insect',
  'monkey',
  'car, auto, automobile, machine, motorcar',
  'feline, felid',
  'truck, motortruck',
  'fruit',
  'fungus',
  'boat'],
 'n_classes': [116, 52, 27, 13, 10, 8, 7, 7, 7, 6, 747, 0, 0],
 'n_classes_frac': array([0.116, 0.052, 0.027, 0.013, 0.01 , 0.008, 0.007, 0.007, 0.007,
        0.006, 0.747, 0.   , 0.   ]),
 'classes': [[151,
   152,
   153,
   154,
   155,
   156,
   157,
   158,
   159,
   160,
   161,
   162,
   163,
   164,
   165,
   166,
   167,
   168,
   169,
   170,
   173,
   174,
   175,
   176,
   177,
   178,
   179,
   180,
   181,
   182,
   183,
   184,
   185,
   186,
   187,
   188,
   189,
   190,
   191,
   192,
   193,
   194,
   195,
   196,
   197,
   198,
   199,
   200,
   201,
   202,
   203,
   204,
   205,
   206,
   207,
   208,
   209,
   210,
   211,
   212,
   213,
   214,
   215,
   216,
   217,
   218,
   219,
   220,
   221,
   222,
   223,
   224,
   225,
   2

: 

In [1]:
class_labels_train = np.load("data/imagenet-embeddings/labels_train.npy")
class_labels_val = np.load("data/imagenet-embeddings/labels_val.npy")

superclass_labels_train = np.vectorize(class_map.get)(class_labels_train)
superclass_labels_val = np.vectorize(class_map.get)(class_labels_val)

np.save("data/imagenet-embeddings/superclass_labels_train.npy", superclass_labels_train)
np.save("data/imagenet-embeddings/superclass_labels_val.npy", superclass_labels_val)

NameError: name 'np' is not defined

In [7]:
# take subset of imagenet embeddings classes 0 - 9
subset_idx_train = np.where(superclass_labels_train < 10)[0]
imagenet_train = np.load("data/imagenet-embeddings/embeddings_vitb4_300ep_train.npy")

subset_idx_val = np.where(superclass_labels_val < 10)[0]
imagenet_val = np.load("data/imagenet-embeddings/embeddings_vitb4_300ep_val.npy")

In [8]:
imagenet_data_subset = imagenet_train[subset_idx_train]
superclass_labels_train_subset = superclass_labels_train[subset_idx_train]

imagenet_data_subset_val = imagenet_val[subset_idx_val]
superclass_labels_val_subset = superclass_labels_val[subset_idx_val]

np.save("data/imagenet-embeddings/embeddings_vitb4_300ep_train_subset.npy", imagenet_data_subset)
np.save("data/imagenet-embeddings/superclass_labels_train_subset.npy", superclass_labels_train_subset)

np.save("data/imagenet-embeddings/embeddings_vitb4_300ep_val_subset.npy", imagenet_data_subset_val)
np.save("data/imagenet-embeddings/superclass_labels_val_subset.npy", superclass_labels_val_subset)

In [7]:
def create_embeddings_dataset():
    '''
    Create torch datasets to be used in a torch Dataloader for training and testing
    '''

    # load embeddings and labels
    embeddings_train = np.load("data/imagenet-embeddings/embeddings_vitb4_300ep_train.npy")
    labels_train = np.load("data/imagenet-embeddings/superclass_labels_train.npy")

    embeddings_val = np.load("data/imagenet-embeddings/embeddings_vitb4_300ep_val.npy")
    labels_val = np.load("data/imagenet-embeddings/superclass_labels_val.npy")

    # convert to torch tensors
    embeddings_train = torch.tensor(embeddings_train)
    labels_train = torch.tensor(labels_train)

    embeddings_val = torch.tensor(embeddings_val)
    labels_val = torch.tensor(labels_val)

    # change labels to long
    labels_train = labels_train.long()
    labels_val = labels_val.long()

    # create datasets with embeddings and labels as
    train_dataset = torch.utils.data.TensorDataset(embeddings_train, labels_train)
    val_dataset = torch.utils.data.TensorDataset(embeddings_val, labels_val)

    train_dataset.targets = labels_train
    val_dataset.targets = labels_val
    
    return train_dataset, val_dataset

In [8]:
train, val = create_embeddings_dataset()